In [2]:
import math
import re
import pandas as pd
import tweepy
import emoji
import numpy as np
# from googleapiclient.discovery import build
from google.cloud import language
from google.cloud.language import types
from google.cloud.language import enums
# import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
date = datetime.today().strftime('%Y-%m-%d')

# authorize tweepy
consumer_key = "OROBANkVOsvva9HETWL4Kovbx"
consumer_secret = "t87379Kk9ANccJDM5a6E6G5eLKDGLnL7s2zld1kXvUCYu9gjnJ"
access_token = "1305689531421786112-FNe1D2tjmvFjzikZYHDyjvRVAVT9gk"
access_token_secret = "G6KV4pfTg6l7BSXxbQZDyYbREmr2taNkYzBzSZZfDyBIa"

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit = True)

languages = ["ar","zh","nl","en","fr","de","id","it","ja","ko","pt","es","th","tr"]

client = language.LanguageServiceClient.from_service_account_json \
    ("/Users/phoenix/Documents/1stSemesterSO/DSCI125/Projects/Data-Science-Final-Project/DSCI_Auth.json")


In [4]:
# HFI Feature Extraction - Capture Step
# ---------------------------------------------------------------------------------------------------------------------
def get_coordinates(filename):
    lat_long = pd.read_csv(filename,usecols = ["latitude","longitude","country"])
    lat_long["country"] = lat_long["country"].astype(str).apply(lambda s: s.strip())

    radius = pd.read_csv("countries of the world.csv",usecols = ["Country","Area (sq. mi.)"])
    radius["Country"] = radius["Country"].astype(str).apply(lambda s: s.strip())
    radius["Area (sq. mi.)"] = radius["Area (sq. mi.)"].astype(float).apply(lambda x: math.sqrt(x))
    radius.columns = ["country","area"]
    radius.columns = ["country","radius"]

    coordinates = pd.merge(lat_long,radius,on = ["country"])

    coordinates["lat-long-radius"] = coordinates["latitude"].astype(str) + "," + \
                                     coordinates["longitude"].astype(str) + \
                                     "," + coordinates["radius"].astype(str) + "mi"

    return coordinates

def read_csv_hfi(filename):
    df_byCountry = pd.read_csv(filename,usecols = ["countries","region","hf_score","hf_rank","hf_quartile",
                                                   "pf_expression","pf_association_assembly","pf_movement"],na_values = ["-"])

    df_byCountry.loc[:,"hf_score":"pf_movement"] = df_byCountry.loc[:,"hf_score":"pf_movement"].astype(float)

    aggregation_functions = {"region": "first","hf_score": "mean","hf_rank": "max","hf_quartile": "max",
                             "pf_expression": "mean","pf_association_assembly": "mean","pf_movement": "mean"}

    df_byRegion = df_byCountry.groupby(by = ["countries"]).aggregate(aggregation_functions)

    return df_byCountry,df_byRegion


# Twitter Interaction and Feature Extraction - Capture Step
# ---------------------------------------------------------------------------------------------------------------------
def filter_regex(text):
    text = re.sub(r"(?:\#|\@|https?\:(\/\/))\S+","",text)
    text = re.sub(r"\w*(RT|rt)\w*","",text)
    text = re.sub(r"(<[^>]+>)\S+","",text)

    text =  emoji.get_emoji_regexp().sub(u'',text)

    return text


# create get_tweets -> takes coordinates, a datetime range to search, result type, maximum number of tweets to scrape
def get_tweets(coordinates,result_type,until_date,count,max_tweets):
    """Takes in 5 parameters, set the max_tweets to 150, count to 25, and result type to 'recent'
    until date as the current date, and the coordinates equal to the 'latitude' and 'longitude'
    values. Returns dataframe with following columns -> text, created_at_date, favorite_count,
    user location, followers count, friends count, and the language used.
    Indexed 0 - 6"""
    tweets = tweepy.Cursor(api.search,geocode = coordinates,result_type = result_type,
                           until = until_date,count = count).items(max_tweets)

    tweets_list = [
         [filter_regex(tweet.text),tweet.created_at,tweet.retweet_count,tweet.favorite_count,
          tweet.user.location,tweet.user.followers_count,tweet.user.friends_count,
          tweet.lang] for tweet in tweets]


    tweets_df = pd.DataFrame(data = tweets_list,columns = ["text","created_at","retweet_count","favorite_count",
                                                           "user_location","followers_count",
                                                           "friends_count","language"])

    return tweets_df


def get_tweets_by_country(coordinates_df):
    """Takes in tweet_df, run the get_tweets to get all those tweets
    Uses latitude, longitude, and radius, return a dataframe with each iteration
    Save into list of dataframes, and merge the list into a single frame
    byCountry -> country, all the columns of tweets_df"""
    # return the dataframe with country, lat, and longitude, radius
    result_type = 'recent'
    max_tweets = 500

    tweets_by_country = pd.DataFrame()

    print(f"retrieving tweets, 5000 tweets remaining")

    # go through the dataframe for the coordinates by country, and retrieve the tweets from each country (~25)
    # save results into dataframe with following characteristics -> country, text
    for coordinate in coordinates_df.itertuples():
        tweets = get_tweets(coordinate[2],result_type,date,150,max_tweets)
        tweets["region"] = coordinate[1]
        tweets_by_country = pd.concat([tweets_by_country,tweets])
        print(f"retrieving tweets, {5000 - len(tweets_by_country)} tweets remaining")

    tweets_by_country = tweets_by_country[tweets_by_country["language"].isin(languages)]

    return tweets_by_country

In [9]:
df,df_aggregates = read_csv_hfi("hfi_cc_2019.csv")
df_by_country,df_by_region = read_csv_hfi("hfi_cc_2019.csv")
df_coordinates = get_coordinates("lat_long_coordinates.csv")

df_regional_coord = pd.read_csv("regional_coordinates .csv")

df_tweets = get_tweets_by_country(df_regional_coord)

retrieving tweets, 5000 tweets remaining
retrieving tweets, 4500 tweets remaining
retrieving tweets, 4000 tweets remaining
retrieving tweets, 3500 tweets remaining
retrieving tweets, 3000 tweets remaining
retrieving tweets, 2500 tweets remaining
retrieving tweets, 2000 tweets remaining
retrieving tweets, 1500 tweets remaining
retrieving tweets, 1000 tweets remaining
retrieving tweets, 500 tweets remaining
retrieving tweets, 0 tweets remaining


In [10]:
# determine Sentiment and Magnitude of Tweets - Process Step$
def find_sentiment_score(tweets_text):

    tweets = pd.DataFrame(columns = ["score","magnitude"])

    document = types.Document(
        content = tweets_text,
        type = enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    response = client.analyze_sentiment(document = document,encoding_type = "UTF8").document_sentiment.score
    
    return response

def find_sentiment_magnitude(tweets_text):

    tweets = pd.DataFrame(columns = ["score","magnitude"])

    document = types.Document(
        content = tweets_text,
        type = enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    response = client.analyze_sentiment(document = document,encoding_type = "UTF8").document_sentiment.magnitude
    
    return response

In [12]:
def build_tweet_dataset(tweets):
    df,df_aggregates = read_csv_hfi("hfi_cc_2019.csv")
    df_by_country,df_by_region = read_csv_hfi("hfi_cc_2019.csv")
    df_coordinates = get_coordinates("lat_long_coordinates.csv")

    df_regional_coord = pd.read_csv("regional_coordinates .csv")

    regions = ['Caucasus & Central Asia', 'East Asia', 'Eastern Europe',
     'Latin America & the Caribbean', 'Middle East & North Africa',
     'North America', 'Oceania', 'South Asia', 'Sub-Saharan Africa',
     'Western Europe']
    
    by_region_dataframe = pd.DataFrame()
    
    tracker = 0
    for r in regions:
        tweet = tweets.loc[tweets["region"] == r].iloc[:100] 
        by_region_dataframe = pd.concat([by_region_dataframe,tweet])
    
    return by_region_dataframe

In [13]:
tweets = build_tweet_dataset(df_tweets)

In [15]:
def find_sentiment_main(tweets):
    
    df_sentiment = tweets.reset_index()

    df_sentiment = df_sentiment.drop([83,460])
    
    df_sentiment["score"] = df_sentiment.text.apply(find_sentiment_score)
    df_sentiment["magnitude"] = df_sentiment.text.apply(find_sentiment_magnitude)

    return df_sentiment


df = find_sentiment_main(tweets)
df

InvalidArgument: 400 The language ku is not supported for document_sentiment analysis.

In [ ]:
df.to_csv("sentiment_analysis_byRegion.csv")

In [ ]:
df